In [1]:
import os
os.chdir(r'D:\BaiduNetdiskDownload\Python\案例\Python评分卡\data')

In [21]:
import pandas as pd
import numpy as np 

In [31]:
#读取数据集，至少包含变量和target两列
sample_set = pd.read_excel('cartdata.xlsx')

In [32]:
sample_set.head()

,Unnamed: 0,target,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,38828,0,0.081455,46,0,1738.000000,NaN,4,0,1,0,0.0
1,29723,0,0.401298,58,1,0.375000,8167.0,6,0,1,0,5.0
2,23178,0,0.214106,62,0,5404.000000,NaN,14,0,2,0,0.0
3,24824,0,0.322189,72,0,1.459318,2875.0,12,0,1,0,0.0
4,15677,0,1.268255,35,2,0.012240,37500.0,5,2,0,3,1.0


In [56]:
sample_set.age.describe()

count    104415.000000
mean         52.363980
std          14.718052
min          21.000000
25%          41.000000
50%          52.000000
75%          63.000000
max         109.000000
Name: age, dtype: float64

In [33]:
sample_set.shape

(104415, 12)

In [34]:
a = sample_set.shape[0] * 0.05
a

5220.75

In [35]:
def calc_score_median(sample_set, var):   
    ''' 
    计算相邻评分的中位数，以便进行决策树二元切分    
    param sample_set: 待切分样本    
    param var: 分割变量名称   
    '''   
    var_list = list(np.unique(sample_set[var]))   
    var_median_list = []    
    for i in range(len(var_list) -1):        
        var_median = (var_list[i] + var_list[i+1]) / 2      
        var_median_list.append(var_median)   
    return var_median_list

In [36]:
calc_score_median(sample_set, 'age')

[21.5,
 22.5,
 23.5,
 24.5,
 25.5,
 26.5,
 27.5,
 28.5,
 29.5,
 30.5,
 31.5,
 32.5,
 33.5,
 34.5,
 35.5,
 36.5,
 37.5,
 38.5,
 39.5,
 40.5,
 41.5,
 42.5,
 43.5,
 44.5,
 45.5,
 46.5,
 47.5,
 48.5,
 49.5,
 50.5,
 51.5,
 52.5,
 53.5,
 54.5,
 55.5,
 56.5,
 57.5,
 58.5,
 59.5,
 60.5,
 61.5,
 62.5,
 63.5,
 64.5,
 65.5,
 66.5,
 67.5,
 68.5,
 69.5,
 70.5,
 71.5,
 72.5,
 73.5,
 74.5,
 75.5,
 76.5,
 77.5,
 78.5,
 79.5,
 80.5,
 81.5,
 82.5,
 83.5,
 84.5,
 85.5,
 86.5,
 87.5,
 88.5,
 89.5,
 90.5,
 91.5,
 92.5,
 93.5,
 94.5,
 95.5,
 96.5,
 97.5,
 98.5,
 100.0,
 101.5,
 102.5,
 104.0,
 107.0]

In [37]:
def choose_best_split(sample_set, var, min_sample):    
    '''    
    使用CART分类决策树选择最好的样本切分点    
    返回切分点    param sample_set: 待切分样本  
    param var: 分割变量名称    
    param min_sample: 待切分样本的最小样本量(限制条件)   
    '''    
    # 根据样本评分计算相邻不同分数的中间值    
    score_median_list = calc_score_median(sample_set, var)    
    median_len = len(score_median_list)   
    sample_cnt = sample_set.shape[0]    
    sample1_cnt = sum(sample_set['target'])    
    sample0_cnt =  sample_cnt- sample1_cnt    
    Gini = 1 - np.square(sample1_cnt / sample_cnt) - np.square(sample0_cnt / sample_cnt)        
    bestGini = 0.0; bestSplit_point = 0.0; bestSplit_position = 0.0   
    for i in range(median_len):        
        left = sample_set[sample_set[var] < score_median_list[i]]        
        right = sample_set[sample_set[var] > score_median_list[i]]                
        left_cnt = left.shape[0]; right_cnt = right.shape[0]       
        left1_cnt = sum(left['target']); right1_cnt = sum(right['target'])       
        left0_cnt =  left_cnt - left1_cnt; right0_cnt =  right_cnt - right1_cnt      
        left_ratio = left_cnt / sample_cnt; right_ratio = right_cnt / sample_cnt              
        if left_cnt < min_sample or right_cnt < min_sample:           
            continue                
        Gini_left = 1 - np.square(left1_cnt / left_cnt) - np.square(left0_cnt / left_cnt)       
        Gini_right = 1 - np.square(right1_cnt / right_cnt) - np.square(right0_cnt / right_cnt)      
        Gini_temp = Gini - (left_ratio * Gini_left + right_ratio * Gini_right)       
        if Gini_temp > bestGini:            
            bestGini = Gini_temp; bestSplit_point = score_median_list[i]          
            if median_len > 1:             
                bestSplit_position = i / (median_len - 1)          
            else:               
                bestSplit_position = i / median_len    
        else:           
            continue                
    Gini = Gini - bestGini  
    return bestSplit_point, bestSplit_position


In [38]:
choose_best_split(sample_set,'age', a)

(55.5, 0.4146341463414634)

In [54]:
def bining_data_split(sample_set, var, min_sample , split_list):  
    '''    
    划分数据找到最优分割点list   
    param sample_set: 待切分样本   
    param var: 分割变量名称   
    param min_sample: 待切分样本的最小样本量(限制条件)   
    param split_list: 最优分割点list    
    '''  
    split, position = choose_best_split(sample_set, var, min_sample)   
    if split != 0.0:        
        split_list.append(split)   
        # 根据分割点划分数据集，继续进行划分    
    sample_set_left = sample_set[sample_set[var] < split]    
    sample_set_right = sample_set[sample_set[var] > split]   
    # 如果左子树样本量超过2倍最小样本量，且分割点不是第一个分割点，则切分左子树   
    if len(sample_set_left) >= min_sample * 2 and position not in [0.0, 1.0]:     
        bining_data_split(sample_set_left, var, min_sample, split_list)  
    else:     
        None   
        # 如果右子树样本量超过2倍最小样本量，且分割点不是最后一个分割点，则切分右子树   
    if len(sample_set_right) >= min_sample * 2 and position not in [0.0, 1.0]:     
        bining_data_split(sample_set_right, var, min_sample, split_list)  
    else:   
        None


In [53]:
def get_bestsplit_list(sample_set, var):  
    '''    
    根据分箱得到最优分割点list   
    param sample_set: 待切分样本   
    param var: 分割变量名称  
    '''    
    # 计算最小样本阈值（终止条件）    
    min_df = sample_set.shape[0] * 0.05   
    split_list = []   
    # 计算第一个和最后一个分割点   
    bining_data_split(sample_set, var, min_df, split_list) 
    return split_list


In [55]:
get_bestsplit_list(sample_set, 'age')

[55.5, 36.5, 29.5, 46.5, 43.5, 40.5, 52.5, 49.5, 62.5, 58.5, 67.5, 74.5]